# Convert photo to cartoon ONNX Model to OpenVINO™ IR

The aim of portrait cartoon stylization is to transform real photos into cartoon images with portrait's ID information and texture details. We use Generative Adversarial Network method to realize the mapping of picture to cartoon. Considering the difficulty in obtaining paired data and the non-corresponding shape of input and output, we adopt unpaired image translation fashion.

The results of CycleGAN, a classic unpaired image translation method, often have obvious artifacts and are unstable. Recently, Kim et al. propose a novel normalization function (AdaLIN) and an attention module in paper "U-GAT-IT" and achieve exquisite selfie2anime results.

Different from the exaggerated anime style, our cartoon style is more realistic and contains unequivocal ID information. To this end, we add a Face ID Loss (cosine distance of ID features between input image and cartoon image) to reach identity invariance.

We propose a Soft Adaptive Layer-Instance Normalization (Soft-AdaLIN) method which fuses the statistics of encoding features and decoding features in de-standardization.

Based on U-GAT-IT, two hourglass modules are introduced before encoder and after decoder to improve the performance in a progressively way.

We also pre-process the data to a fixed pattern to help reduce the difficulty of optimization. For details, see below.

This tutorial demonstrates the running results of photo2cartoon and converts it from ONNX model to the IR model used by OpenVION.

Requirements

    ·python 3.6
    ·pytorch 1.4
    ·tensorflow-gpu 1.14
    ·face-alignment
    ·dlib
    ·onnxruntime


## Imports

The default environment of  openvino does not contain all the packages required for the current model to run, so you need to run the following command first.

In [ ]:
%pip install face-alignment dlib onnxruntime

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from pathlib import Path

## Prerequisittes

Pytorch Model is not in Gihub, so if you want pre-dowload it in: [Google Drive](https://drive.google.com/file/d/1PhwKDUhiq8p-UqrfHCqj257QnqBWD523/view?usp=sharing) or [Baidu Cloud](https://pan.baidu.com/share/init?surl=MsT3-He3UGipKhUi4OcCJw) acess code: y2ch. 

Now we clone photo2cartoon project.

In [ ]:
if not Path('photo2cartoon').exists():
    !git clone https: // github.com/sususama/photo2cartoon.git
%cd photo2cartoon

## Check model inference

`test_onnx.py` script run ONNXmodel to test photo to cartoon.

Please use a young Asian woman photo.

In [ ]:
!python test_onnx.py --photo_path images/photo_test.jpg --save_path images/cartoon_result.png

In [ ]:
original_img = Image.open('images/photo_test.jpg')
generate_img = Image.open('images/cartoon_result.png')
fig, ax = plt.subplots(1, 2)
ax[0].imshow(original_img)
ax[0].set_title('original')
ax[1].imshow(generate_img)
ax[1].set_title('generate')
plt.show()

This part show how to use the ONNX Model to generate cartoon image.

This open source model is based on the trainning of yong women in Asia.For other groups with insufficient coverage, you can collect the data of corresponding groups according to the use scenario for training.

## Conver ONNX Model to OpenVINO Intermediate Representation (IR)

In [ ]:
from openvino.tools import mo
from openvino.runtime import serialize

model = mo.convert_model('models/photo2cartoon_weights.onnx')
serialize(model, 'models/photo2cartoon.xml')

## Verify model inference

To infer the model, you first need to call the `ExecutableNetwork` method `create_infer_Request()` to create an inference request, we use `compile_Model()` loaded `exec_net`.  Then we must call `infer()` as `_InferRequest_`  The method requires a parameter: `inputs`. This is a dictionary that maps input layer names to input data.

- Step 1: Import the model. We passed `ie.read_Model` to read the model, `ie.confile_Model` to compile the model.

In [ ]:
from utils.preprocess import Preprocess
from openvino.runtime import Core

ie = Core()
pre = Preprocess()
print("Load network")
photo2cartoon_model_xml = "models/photo2cartoon.xml"
model = ie.read_model(model=photo2cartoon_model_xml)
compiled_model = ie.compile_model(model=model, device_name="CPU")
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)
print('Model Input and Output Info')
print(f"- input shape: {input_layer.shape}")
print(f"- input precision: {input_layer.element_type}")
print(f"- output shape: {output_layer.shape}")
print(f"- output precision: {output_layer.element_type}")

- Step 2: Load the image, preprocess it, and convert it to the input shape. To propagate an image through the network, you need to load it into an array, resize it to the shape the network expects, and convert it to the network's input layout format. We get a reference of the required height and width for the web and resize the image to this size. Finally, we resize the image to N, C, H, W format (where N=1), first calling `np.transpose()` to change to C, H, W.

In [ ]:
print("Load input image")
image_filename = 'images/photo_test.jpg'
image = cv2.cvtColor(cv2.imread(image_filename), cv2.COLOR_BGR2RGB)
face_rgba = pre.process(image)
face_rgba = cv2.resize(face_rgba, (256, 256), interpolation=cv2.INTER_AREA)
face = face_rgba[:, :, :3].copy()
mask = face_rgba[:, :, 3][:, :, np.newaxis].copy() / 255
image = (face * mask + (1 - mask) * 255) / 127.5 - 1
print("- input image shape: {}".format(image.shape))
N, C, H, W = input_layer.shape
resized_image = cv2.resize(src=image, dsize=(W, H))
print("- resize image into shape: {}".format(resized_image.shape))
input_data = np.expand_dims(np.transpose(
    resized_image, (2, 0, 1)), 0).astype(np.float32)
print("- align image shape same as network input: {}".format(input_data.shape))

- Step 3: Model reasoning. We can use `compiled_Model([input_data])[output_layer]` directly obtains the result of reasoning.

In [ ]:
print("Infrence")
result = compiled_model([input_data])[output_layer]
print("- generate image[0] shape: {}".format(result[0].shape))
print("- generate image precision: {}".format(result.dtype))
result_path = 'images/openvino_result.jpg'
generate_image = result[0].transpose((1, 2, 0))
cv2.imwrite(result_path, generate_image)
plt.imshow(generate_image)
plt.show()